In [1]:
import pandas as pd
import numpy as np
import os
from scipy import stats

In [2]:
df = pd.read_csv('State_of_data_2022.csv')

In [7]:
df.shape

(4271, 353)

In [8]:
df.head()

,"('P0', 'id')","('P1_a ', 'Idade')","('P1_a_1 ', 'Faixa idade')","('P1_b ', 'Genero')","('P1_c ', 'Cor/raca/etnia')","('P1_d ', 'PCD')","('P1_e ', 'experiencia_profissional_prejudicada')","('P1_f ', 'aspectos_prejudicados')","('P1_g ', 'vive_no_brasil')","('P1_i ', 'Estado onde mora')",...,"('P8_d_2 ', 'Coletando e limpando os dados que uso para análise e modelagem.')","('P8_d_3 ', 'Entrando em contato com os times de negócio para definição do problema, identificar a solução e apresentação de resultados.')","('P8_d_4 ', 'Desenvolvendo modelos de Machine Learning com o objetivo de colocar em produção em sistemas (produtos de dados).')","('P8_d_5 ', 'Colocando modelos em produção, criando os pipelines de dados, APIs de consumo e monitoramento.')","('P8_d_6 ', 'Cuidando da manutenção de modelos de Machine Learning já em produção, atuando no monitoramento, ajustes e refatoração quando necessário.')","('P8_d_7 ', 'Realizando construções de dashboards em ferramentas de BI como PowerBI, Tableau, Looker, Qlik, etc.')","('P8_d_8 ', 'Utilizando ferramentas avançadas de estatística como SAS, SPSS, Stata etc, para realizar análises.')","('P8_d_9 ', 'Criando e dando manutenção em ETLs, DAGs e automações de pipelines de dados.')","('P8_d_10 ', 'Criando e gerenciando soluções de Feature Store e cultura de MLOps.')","('P8_d_11 ', 'Criando e mantendo a infra que meus modelos e soluções rodam (clusters, servidores, API, containers, etc.)')"
0,zzqzz3l9ily8nuo2m7wyzzqzz3w48o96,39.0,35-39,Masculino,Parda,Não,Não acredito que minha experiência profissiona...,NaN,True,Distrito Federal (DF),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,zzls2oftfn9law393oezzls2ofhvfpzd,32.0,30-34,Masculino,Parda,Não,"Sim, acredito que a minha a experiência profis...",Aprovação em processos seletivos/entrevistas,True,Pará (PA),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,zzdwqzfqqp1ypc7ps6m0hzzdwqz292yi,53.0,50-54,Masculino,Branca,Não,NaN,NaN,True,Distrito Federal (DF),...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,zzbqh3uy7yk7k9qmkzzbqtb4s9faqspl,27.0,25-29,Masculino,Branca,Não,NaN,NaN,True,Minas Gerais (MG),...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,zzaf1m95yan929rb94wzzaf1mekhvhpg,46.0,45-49,Feminino,Branca,Não,Não acredito que minha experiência profissiona...,NaN,True,Pará (PA),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df.columns[16:50]

Index(['('P2_a ', 'Qual sua situação atual de trabalho?')',
       '('P2_b ', 'Setor')', '('P2_c ', 'Numero de Funcionarios')',
       '('P2_d ', 'Gestor?')', '('P2_e ', 'Cargo como Gestor')',
       '('P2_f ', 'Cargo Atual')', '('P2_g ', 'Nivel')',
       '('P2_h ', 'Faixa salarial')',
       '('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')',
       '('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')',
       '('P2_k ', 'Você está satisfeito na sua empresa atual?')',
       '('P2_l ', 'Qual o principal motivo da sua insatisfação com a empresa atual?')',
       '('P2_l_1 ', 'Falta de oportunidade de crescimento no emprego atual')',
       '('P2_l_2 ', 'Salário atual não corresponde ao mercado')',
       '('P2_l_3 ', 'Não tenho uma boa relação com meu líder/gestor')',
       '('P2_l_4 ', 'Gostaria de trabalhar em em outra área de atuação')',
       '('P2_l_5 ', 'Gostaria de receber ma

In [46]:
p1 = ["('P0', 'id')", "('P1_a ', 'Idade')", "('P1_a_1 ', 'Faixa idade')","('P1_b ', 'Genero')", "('P1_c ', 'Cor/raca/etnia')","('P1_g ', 'vive_no_brasil')","('P1_i_2 ', 'Regiao onde mora')",
      "('P1_m ', 'Área de Formação')","('P2_d ', 'Gestor?')"]
duvidas_p1 = ["('P1_m ', 'Área de Formação')","('P2_a ', 'Qual sua situação atual de trabalho?')","('P2_b ', 'Setor')", "('P2_c ', 'Numero de Funcionarios')", ""]

In [ ]:
p2 = []
duvidas_p2 = ["('P2_a ', 'Qual sua situação atual de trabalho?')","('P2_b ', 'Setor')", "('P2_c ', 'Numero de Funcionarios')", "('P2_d ', 'Gestor?')"]

In [32]:
df1 = df[p0]

In [52]:
df["('P2_g ', 'Nivel')"].value_counts()

Pleno     1060
Júnior    1023
Sênior     898
Name: ('P2_g ', 'Nivel'), dtype: int64